In [1]:
# only run once
#!conda create -n resnet_fl_2host python=3 anaconda -y

In [1]:
# activate conda environment to access pysyft
!source /usr/local/anaconda3/bin/activate resnet_fl_2host

In [18]:
!pip install natsort

In [1]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import syft as sy
import numpy as np
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy
from PIL import Image
import glob 

In [2]:
hook = sy.TorchHook(torch)

In [3]:
class Arguments():
    def __init__(self):
        self.batch_size = 4
        self.test_batch_size = 100
        self.epochs = 5
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = True

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

torch.set_num_threads(4)

In [4]:
import pandas as pd

df = pd.read_excel ('../multilabels/LandUse_Multilabeled.xlsx')
df_label = np.array(df)

In [5]:
class_names =  np.array(["airplane","bare-soil","buildings","cars","chaparral","court","dock","field","grass","mobile-home","pavement","sand","sea","ship","tanks","trees","water"])

In [6]:
largestxor = 0 
largestij = (0,0)

for i in range(1,17):
    for j in range(i+1,18):
        #colnand = np.sum(np.logical_not(np.logical_and(df_label[:,i], df_label[:,j])))
        colxor = np.sum(np.logical_xor(df_label[:,i].astype(bool) , df_label[:,j].astype(bool) )) -  np.sum(np.logical_and(df_label[:,i], df_label[:,j]))
        #print(i,j, colxor, colnand)
        if colxor >= largestxor and np.sum(df_label[:,i]) >=700 and np.sum(df_label[:,j])>= 700 :
            largestxor = colxor
            largestij = (i,j)
print(largestxor,class_names[largestij[0]-1], class_names[largestij[1]-1], largestij)

674 bare-soil cars (2, 4)


In [7]:
def uncor_selecter(nr_label = 4,min_img = 300):
    """retrun a list with the least correlated labels """
    image_perlabel = np.sum(df_label[:,1:],axis= 0)
    biggest_label =np.where(np.any([image_perlabel > min_img],axis=0))[0]
    #print(biggest_label, image_perlabel[biggest_label])

    selected_list = [] 
    allcor_lost = np.array([0,0,0])
    for i in range(0,len(biggest_label)-1):
        it = biggest_label[i]
        for j in range(i+1,len(biggest_label)):
            jt = biggest_label[j]

            colxor = np.sum(np.logical_xor(df_label[:,it].astype(bool) , df_label[:,jt].astype(bool) )) -  np.sum(np.logical_and(df_label[:,it], df_label[:,jt]))
            allcor_lost = np.vstack((allcor_lost, np.array([colxor,it,jt]))) 
    sorted_list = allcor_lost[allcor_lost[:,0].argsort()]
    selected_list.append(sorted_list[-1,1])
    selected_list.append(sorted_list[-1,2])
    #print(sorted_list, selected_list)        

    while len(selected_list)<nr_label:
        biggest_label = np.setdiff1d(biggest_label,np.array(selected_list))
        largestxor = 0 
        largestind = 0
        for i in biggest_label:
            overall_xor = 0 
            for j in (selected_list):
                overall_xor += np.sum(np.logical_xor(df_label[:,i].astype(bool) , df_label[:,j].astype(bool) )) -  np.sum(np.logical_and(df_label[:,i], df_label[:,j]))

            if overall_xor >= largestxor:
                largestxor = overall_xor
                largestind = i

        selected_list.append(largestind)
    
    return selected_list

In [8]:
import random

def sampler_split_for_client(cdata, idxs, nr_client=4, minimum_skew_percentage = .2):
    selected_labels = uncor_selecter(nr_client,300)
    
    splitlists = []
    for sb in selected_labels:
        splitlists.append([])
        
    
    for i in idxs:
        nplabel = cdata.__getlabel__(i)
        #nplabel = label.numpy()
        
        if np.any(nplabel[selected_labels] == 1):
            if random.random() < minimum_skew_percentage:
                
                flip = np.random.randint(np.sum(nplabel[selected_labels] == 1)) 
                mask = np.where(nplabel[selected_labels] == 1)[0][flip]
                splitlists[mask].append(i)
            
            else:
                flip = np.random.randint(nr_client) 
                splitlists[flip].append(i)
                    
        else:
            flip = np.random.randint(nr_client) 
            splitlists[flip].append(i)

    
    for alist in splitlists:
        print(len(alist))
    return splitlists
    

In [9]:
from torch.utils.data import Dataset
from natsort import natsorted

class CustomDataSet(Dataset):
    def __init__(self, main_dir, transform, labelmat):
        self.main_dir = main_dir
        self.transforms = transform
        self.all_imgs = glob.glob(os.path.join(main_dir, '**/*.tif'), recursive=True)
        self.total_imgs = natsorted(self.all_imgs)
        self.xlabels = labelmat
        
    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        #print(idx,len(self.total_imgs))
        img_loc = self.total_imgs[idx]
        #print(img_loc)
        imagebaselabel = os.path.splitext(os.path.basename(img_loc))[0]
        label = self.xlabels[np.where(self.xlabels[:,0] == imagebaselabel),1:].reshape(17).astype(np.int64)
        #print(label,label.shape)
        tensor_label =  torch.from_numpy(label)
        image = Image.open(img_loc).convert("RGB")
        tensor_image = self.transforms(image)
        return tensor_image, tensor_label
    
    def __getlabel__(self, idx):
        
        img_loc = self.total_imgs[idx]
        #print(img_loc)
        imagebaselabel = os.path.splitext(os.path.basename(img_loc))[0]
        label = self.xlabels[np.where(self.xlabels[:,0] == imagebaselabel),1:].reshape(17).astype(np.int64)
        
        return label

In [18]:
data_dir = "../UCMerced_LandUse/Images"

def load_split_train_test(datadir, labelmat, valid_size=.2, num_clients=4):
    train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    test_transforms = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    train_data = CustomDataSet(datadir, transform=train_transforms, labelmat=labelmat)
    test_data = CustomDataSet(datadir, transform=train_transforms, labelmat=labelmat)

    indices = list(range(2100))
    split = int(np.floor(valid_size * 2100))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    
    lists = sampler_split_for_client(train_data, train_idx, num_clients, .4)
    
    # test dataset
    test_sampler = SubsetRandomSampler(test_idx)
    test_loader = torch.utils.data.DataLoader(test_data, sampler=test_sampler, batch_size=4)
    test_loader_dict = { 'data': test_loader, 'size': len(test_sampler) }
    
    dataloaders = []
    for client_sampler in lists:
        train_sampler = SubsetRandomSampler(client_sampler)
        train_loader = torch.utils.data.DataLoader(
            train_data,
            sampler=train_sampler,
            batch_size=args.batch_size
        )
        dataloaders.append( {'data': train_loader, 'size': len(client_sampler)} )
    
    return dataloaders, test_loader_dict, len(train_idx)

In [67]:
clients_listo, valloader, train_len = load_split_train_test(data_dir, df_label, 0.05, 5)

356
350
397
410
482


In [68]:
def train_client_model(args, model, device, client_dataloader, optimizer, criterion, scheduler, local_epochs):
    
    # train
    for epoch in range(local_epochs):
        
        running_loss_train, running_loss_val = 0, 0
        running_corrects_train, running_corrects_val = 0, 0
        
        # set model to training mode
        model.train()

        for data, target in client_dataloader['data']:
            
            data = data.to(device)
            target = target.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()
            
            # forward
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            
            running_loss_train += loss.item() * args.batch_size
            running_corrects_train += torch.sum(preds == labels.data)
            
        epoch_loss_train = running_loss_train / client_dataloader['size']
        epoch_acc_train = running_corrects_train.double() /  client_dataloader['size']
            
    return model

In [69]:
def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=5, phase = 'train'):
    tloss, tacc= [] , []
    vloss, vacc= [] , []
    
    since = time.time()

    #best_model_wts = copy.deepcopy(model.state_dict())
    #best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        if True:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders['data']:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    #_, preds = torch.max(outputs, 1)
                    outputcpu = outputs.cpu()
                    preds = np.heaviside(outputcpu.detach().numpy(),0)
                    #print(outputs, preds)
                    loss = criterion(outputs, labels.type(torch.float))

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                    #outputsnp = outputs.cpu().numpy()
                    #preds = np.array(outputsnp > 0.5, dtype=float)

                running_loss += loss.item() * inputs.size(0)
                running_corrects += ((torch.sum(torch.from_numpy(preds).to(device) == labels.data)).item() / len(class_names))
                #print("running_corrects",running_corrects)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataloaders['size']
            epoch_acc = (running_corrects) / dataloaders['size']
            
            if phase == 'train':
                tloss.append(epoch_loss)
                tacc.append(epoch_acc)
            
            if phase == 'val':
                vloss.append(epoch_loss)
                vacc.append(epoch_acc)
            
            #print(dataset_sizes[phase],epoch_acc)
            #print(type(epoch_loss),type(epoch_acc))
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    #print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    #model.load_state_dict(best_model_wts)
    return copy.deepcopy(model),[tloss,tacc,vloss,vacc]

In [85]:
LOCAL_EPOCHS = 5
C_FRACTION = 0.7

#clients is array of dataloaders
def train_fedavg_model(model, device, clients, optimizer, criterion, scheduler, c_fraction, epochs=10):
    
    # initial model
    init_model = copy.deepcopy(model)
    
    # iterate through epochs
    for i in range(epochs):
        # get random subset of clients
        fraction = int( c_fraction * float(len(clients)) )
        client_subset = random.sample(clients, fraction)
        
        # train each of the clients
        model_client_list = []
        print("Running epoch numero " + str(i))
        for client in client_subset:
            model_for_client = copy.deepcopy(model)
            client_model, statistics = train_model(model_for_client, client, criterion, optimizer, scheduler, num_epochs=3, phase = 'train')
            model_client_list.append(client_model)
            print("Done with clientelo numero x with stats: ", statistics)
            
        # first initializer
        model_state = model_client_list[0].state_dict()
        client_data_size = client_subset[0]['size']
        for key in model_state:
            model_state[key] = (client_data_size / train_len) * model_state[key]
        
        for c in range(1, len(model_client_list)):
            
            client_model_state = model_client_list[c].state_dict()
            print("client model", client_model_state)
            print("init model", init_model.state_dict())
            client_new_data_size = client_subset[c]['size']
            
            for key in model_state:
                model_state[key] += (client_new_data_size / train_len) * client_model_state[key]
                
        averagedModel = copy.deepcopy(init_model)
        averagedModel.load_state_dict(model_state)
        
        model = copy.deepcopy(averagedModel)
        #print(model.numpy())
        #diff = model.numpy() - init_model.numpy()
        #print(diff)
        model, statistics = train_model(model, valloader, criterion, optimizer, scheduler, num_epochs=1, phase = 'val')
        print("Done with validation", statistics)
    
    return model    

In [86]:
class LENET(nn.Module):
    def __init__(self, n_classes):
        super(LENET, self).__init__()
        from collections import OrderedDict
        self.conv1 = nn.Conv2d(3, 16, kernel_size=(5, 5))
        self.conv2 = nn.Conv2d(16, 32, kernel_size=(5, 5))
        self.conv3 = nn.Conv2d(32, 64, kernel_size=(5, 5))
        #self.conv4 = nn.Conv2d(64, 128, kernel_size=(5, 5))
        self.linear1 = nn.Linear(64 * 24 * 24, 120)
        self.linear2 = nn.Linear(120, 84)
        self.linear3 = nn.Linear(84, n_classes)                                
    def forward(self, x):
        """
        Args:
          x of shape (batch_size, 1, 28, 28): Input images.
        
        Returns:
          y of shape (batch_size, 10): Outputs of the network.
        """
        x = F.max_pool2d(F.relu(self.conv1(x)), kernel_size=2, stride=2)
        x = F.max_pool2d(F.relu(self.conv2(x)), kernel_size=2, stride=2)
        x = F.max_pool2d(F.relu(self.conv3(x)), kernel_size=2, stride=2)
        #x = F.max_pool2d(F.relu(self.conv4(x)), kernel_size=2, stride=2)
        x = x.view(-1, 64 * 24 * 24)
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x

In [87]:
from torchvision import models
from torch.optim import lr_scheduler
import time
import os
import copy

In [88]:
model = LENET(len(class_names))
model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer_ft = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [89]:
model = train_fedavg_model(model, device, clients_listo, optimizer_ft, criterion, exp_lr_scheduler, C_FRACTION, 20)

Running epoch numero 0
Epoch 0/2
----------
train Loss: 0.6916 Acc: 0.5143
Epoch 1/2
----------
train Loss: 0.6916 Acc: 0.5143
Epoch 2/2
----------
train Loss: 0.6916 Acc: 0.5143
Training complete in 0m 6s
Done with clientelo numero x with stats:  [[0.6916277106823189, 0.6916465137707496, 0.6916363299343388], [0.514298414579938, 0.514298414579938, 0.514298414579938], [], []]
Epoch 0/2
----------
train Loss: 0.6918 Acc: 0.5176
Epoch 1/2
----------
train Loss: 0.6918 Acc: 0.5176
Epoch 2/2
----------
train Loss: 0.6918 Acc: 0.5176
Training complete in 0m 7s
Done with clientelo numero x with stats:  [[0.6917727728107658, 0.6917629073764279, 0.6917609542731922], [0.5175738345130585, 0.5175738345130586, 0.5175738345130587], [], []]
Epoch 0/2
----------
train Loss: 0.6914 Acc: 0.5124
Epoch 1/2
----------
train Loss: 0.6914 Acc: 0.5124
Epoch 2/2
----------
train Loss: 0.6914 Acc: 0.5124
Training complete in 0m 6s
Done with clientelo numero x with stats:  [[0.6913700485497378, 0.691356583927454

client model OrderedDict([('conv1.weight', tensor([[[[-2.8771e-02, -1.0189e-01,  5.8947e-03, -1.7585e-02,  5.0812e-02],
          [ 3.7171e-02,  7.8360e-02, -1.0661e-01,  5.5621e-02, -6.8028e-02],
          [ 1.3880e-02,  4.5303e-02, -1.0651e-01,  5.3555e-02,  1.0255e-01],
          [-7.5161e-02, -3.8964e-02, -8.8608e-02, -9.1872e-04,  4.0314e-02],
          [ 5.4479e-02,  8.4655e-02,  2.4184e-03, -4.8115e-02,  4.9557e-02]],

         [[ 4.6163e-02,  6.6902e-04, -3.0194e-02, -8.3957e-02, -5.5819e-02],
          [ 2.4165e-02, -9.7089e-03,  1.8341e-02, -9.1572e-02, -1.1123e-01],
          [-9.6200e-02,  8.5516e-02, -1.0148e-01,  1.0637e-01, -1.5110e-02],
          [-9.7455e-02, -7.1917e-02,  6.0245e-02, -1.3178e-02, -1.4186e-03],
          [ 5.4408e-04, -6.8307e-02,  5.5946e-02,  9.7775e-02,  4.6027e-02]],

         [[ 2.4112e-02,  8.4295e-02, -2.6535e-02, -9.4258e-02,  2.1705e-02],
          [ 4.3502e-02,  1.1106e-01,  7.3362e-03, -7.3715e-02,  7.1644e-02],
          [-4.4517e-03, -8.61

val Loss: 0.6921 Acc: 0.5266
Training complete in 0m 0s
Done with validation [[], [], [0.6921096058118911], [0.526610644257703]]
Running epoch numero 1
Epoch 0/2
----------
train Loss: 0.6922 Acc: 0.5142
Epoch 1/2
----------
train Loss: 0.6922 Acc: 0.5142
Epoch 2/2
----------
train Loss: 0.6922 Acc: 0.5142
Training complete in 0m 6s
Done with clientelo numero x with stats:  [[0.6922076667227396, 0.6922091219483353, 0.6922083116159207], [0.5142037302725968, 0.514203730272597, 0.514203730272597], [], []]
Epoch 0/2
----------
train Loss: 0.6925 Acc: 0.5176
Epoch 1/2
----------
train Loss: 0.6925 Acc: 0.5176
Epoch 2/2
----------
train Loss: 0.6925 Acc: 0.5176
Training complete in 0m 7s
Done with clientelo numero x with stats:  [[0.6924543200192115, 0.6924547696509302, 0.6924540632987912], [0.5175738345130587, 0.5175738345130587, 0.5175738345130587], [], []]
Epoch 0/2
----------
train Loss: 0.6922 Acc: 0.5124
Epoch 1/2
----------
train Loss: 0.6922 Acc: 0.5124
Epoch 2/2
----------
train Los

client model OrderedDict([('conv1.weight', tensor([[[[-1.7811e-02, -6.3077e-02,  3.6491e-03, -1.0886e-02,  3.1455e-02],
          [ 2.3011e-02,  4.8508e-02, -6.5998e-02,  3.4432e-02, -4.2113e-02],
          [ 8.5923e-03,  2.8045e-02, -6.5936e-02,  3.3153e-02,  6.3483e-02],
          [-4.6528e-02, -2.4121e-02, -5.4853e-02, -5.6873e-04,  2.4956e-02],
          [ 3.3725e-02,  5.2405e-02,  1.4971e-03, -2.9785e-02,  3.0678e-02]],

         [[ 2.8577e-02,  4.1416e-04, -1.8691e-02, -5.1973e-02, -3.4555e-02],
          [ 1.4959e-02, -6.0103e-03,  1.1354e-02, -5.6688e-02, -6.8859e-02],
          [-5.9552e-02,  5.2939e-02, -6.2823e-02,  6.5850e-02, -9.3539e-03],
          [-6.0329e-02, -4.4520e-02,  3.7295e-02, -8.1580e-03, -8.7820e-04],
          [ 3.3681e-04, -4.2285e-02,  3.4633e-02,  6.0527e-02,  2.8493e-02]],

         [[ 1.4926e-02,  5.2182e-02, -1.6427e-02, -5.8350e-02,  1.3436e-02],
          [ 2.6930e-02,  6.8751e-02,  4.5415e-03, -4.5633e-02,  4.4351e-02],
          [-2.7558e-03, -5.33

val Loss: 0.6924 Acc: 0.5266
Training complete in 0m 0s
Done with validation [[], [], [0.6924142593429202], [0.526610644257703]]
Running epoch numero 2
Epoch 0/2
----------
train Loss: 0.6926 Acc: 0.5176
Epoch 1/2
----------


KeyboardInterrupt: 